# 2 Tissue Compartment Kinetic Modeling Demo

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sympy import sympify, symbols, lambdify, integrate

## Setup of a (poly-exponential) arterial input function

In [ ]:
# amplitudes and exponents for the poly exponential AIF

# select the type of AIF to use
# choices are 1: "bolus", 2: "constant infusion", 3: "constant infusion and bolus"
AIF_type = 1

if AIF_type == 1:
    AIF_amps = [-2, 1.5, 0.5]
    AIF_exps = [4, 1, 0.01]
elif AIF_type == 2:
    AIF_amps = [-2, 1.5, 0.5]
    AIF_exps = [40, 40, 0]
elif AIF_type == 3:
    AIF_amps = [-15, 14.5, 0.5]
    AIF_exps = [4, 1, 0]
else:
    raise ValueError("Unknown AIF type")

t = symbols("t")

# array of time points (minutes) for visualization
t_values = np.linspace(0, 120, 1000)  # 500 points between 0 and 10

# setup the arterial input function (sum of exponentials)

AIF_str = "+".join(
    [f"{AIF_amp}*exp(-{AIF_exp}*t)" for AIF_amp, AIF_exp in zip(AIF_amps, AIF_exps)]
)

AIF_func = sympify(AIF_str)
AIF_func_numeric = lambdify(t, AIF_func, modules=["numpy"])

fig1, ax1 = plt.subplots(figsize=(4, 3), tight_layout=True)
ax1.plot(t_values, AIF_func_numeric(t_values), "k")
ax1.set_xlabel("t (min)")
ax1.set_ylabel("Ca(t)")
ax1.grid(ls=":")

## Input of the model parameters

In [ ]:
# rate constants of the 2 tissue compartment model
# unit [1/min]

K1 = 0.1
k2 = 0.12
k3 = 0.05
k4 = 0


## Calculation of the unit impulse reponses

In [ ]:
# calculate amplitudes and exponents for the united impulse response function
a1 = (k2 + k3 + k4 - np.sqrt((k2 + k3 + k4) ** 2 - 4 * k2 * k4)) / 2
a2 = (k2 + k3 + k4 + np.sqrt((k2 + k3 + k4) ** 2 - 4 * k2 * k4)) / 2

# calculate the united impulse response function for the 1st and 2nd tissue compartment
UIR1_exps = [a1, a2]
UIR1_amps = [
    K1 * (k4 - a1) / (a2 - a1),
    K1 * (a2 - k4) / (a2 - a1),
]

UIR2_exps = [a1, a2]
UIR2_amps = [
    K1 * k3 / (a2 - a1),
    -K1 * k3 / (a2 - a1),
]

# setup the united impulse response functions for the 1st and 2nd tissue compartment
UIR1_str = "+".join(
    [f"{UIR_amp}*exp(-{UIR_exp}*t)" for UIR_amp, UIR_exp in zip(UIR1_amps, UIR1_exps)]
)

UIR1_func = sympify(UIR1_str)
UIR1_func_numeric = lambdify(t, UIR1_func, modules=["numpy"])

UIR2_str = "+".join(
    [f"{UIR_amp}*exp(-{UIR_exp}*t)" for UIR_amp, UIR_exp in zip(UIR2_amps, UIR2_exps)]
)

UIR2_func = sympify(UIR2_str)
UIR2_func_numeric = lambdify(t, UIR2_func, modules=["numpy"])

# calculate the sum of the two impulse response functions
UIR_func = UIR1_func + UIR2_func
UIR_func_numeric = lambdify(t, UIR_func, modules=["numpy"])

print(f"IRF of 1st  TC: {UIR1_func}")
if k3 != 0:
    print(f"IRF of 2nd  TC: {UIR2_func}")
    print(f"IRF of both TC: {UIR_func}")


## Calculation of the system response to the input function

In [ ]:
# calculate the response of the 1st compartment
resp1_str_list = []

for (UIR_amp, UIR_exp), (AIF_amp, AIF_exp) in itertools.product(
    zip(UIR1_amps, UIR1_exps), zip(AIF_amps, AIF_exps)
):
    if UIR_exp == AIF_exp:
        tmp = f"{AIF_amp} * {UIR_amp} * t * exp(-{UIR_exp}*t)"
    else:
        tmp = f"({AIF_amp} * {UIR_amp}) * (exp(-{UIR_exp}*t) - exp(-{AIF_exp}*t)) / ({AIF_exp} - {UIR_exp})"

    resp1_str_list.append(tmp)

resp1_str = "+".join(resp1_str_list)
resp1_func = sympify(resp1_str)
resp1_func_numeric = lambdify(t, resp1_func, modules=["numpy"])

print(f"response of the 1st TC C1(t) : {resp1_func}")

# calculate the response of the 2nd compartment
if k3 != 0:
    resp2_str_list = []

    for (UIR_amp, UIR_exp), (AIF_amp, AIF_exp) in itertools.product(
        zip(UIR2_amps, UIR2_exps), zip(AIF_amps, AIF_exps)
    ):
        if UIR_exp == AIF_exp:
            tmp = f"{AIF_amp} * {UIR_amp} * t * exp(-{UIR_exp}*t)"
        else:
            tmp = f"({AIF_amp} * {UIR_amp}) * (exp(-{UIR_exp}*t) - exp(-{AIF_exp}*t)) / ({AIF_exp} - {UIR_exp})"

        resp2_str_list.append(tmp)

    resp2_str = "+".join(resp2_str_list)
    resp2_func = sympify(resp2_str)
    resp2_func_numeric = lambdify(t, resp2_func, modules=["numpy"])

    # calculate the total response of the system
    resp_func = resp1_func + resp2_func
    resp_func_numeric = lambdify(t, resp_func, modules=["numpy"])

    print(f"response of the 2nd TC C2(t) : {resp2_func}")
    print(f"response of the C1(t) + C2(t): {resp_func}")
else:
    resp_func = resp1_func

## Visualization of results

In [ ]:
# Plot the function


fig, ax = plt.subplots(3, 1, figsize=(7, 10), tight_layout=True)
if k3 != 0:
    ax[0].plot(
        t_values,
        UIR1_func_numeric(t_values),
        "--",
        label="IR C1",
        color=plt.cm.tab10(1),
    )
    ax[0].plot(
        t_values, UIR2_func_numeric(t_values), ":", label="IR C2", color=plt.cm.tab10(2)
    )
    ax[0].plot(
        t_values,
        UIR_func_numeric(t_values),
        label="IR (C1 + C2)",
        color=plt.cm.tab10(0),
    )
else:
    ax[0].plot(
        t_values, UIR1_func_numeric(t_values), label="IR C1", color=plt.cm.tab10(0)
    )


ax[0].legend()
ax[0].set_ylabel("IR(t)")
ax[0].set_ylim(0, None)
ax[0].set_title(
    f"impulse responses for K1={K1} 1/min, k2={k2} 1/min, k3={k3} 1/min, k4={k4} 1/min",
    fontsize="medium",
)

ax[1].plot(t_values, AIF_func_numeric(t_values), "k")
ax[1].set_ylabel("Ca(t)")
ax[1].set_title(f"arterial input function", fontsize="medium")

if k3 != 0:
    ax[2].plot(
        t_values, resp1_func_numeric(t_values), "--", label="C1", color=plt.cm.tab10(1)
    )
    ax[2].plot(
        t_values, resp2_func_numeric(t_values), ":", label="C2", color=plt.cm.tab10(2)
    )
    ax[2].plot(
        t_values, resp_func_numeric(t_values), label="C1 + C2", color=plt.cm.tab10(0)
    )
else:
    ax[2].plot(
        t_values, resp1_func_numeric(t_values), label="C1", color=plt.cm.tab10(0)
    )
ax[2].legend()
ax[2].set_ylabel("C(t)")
ax[2].set_title(f"system responses", fontsize="medium")

for axx in ax.ravel():
    axx.set_xlabel("t [min]")
    axx.grid(ls=":")


# ## Logan plot

In [ ]:

# add time points for 5min frames
frame_time = 5
num_frames = t_values.max() // frame_time
frame_times = frame_time * np.arange(1, num_frames)

time_integrated_resp_func = integrate(resp_func, (t, 0, t))
time_integrated_aif = integrate(AIF_func, (t, 0, t))

if k3 != 0 and k4 == 0:
    patlak_t_func = time_integrated_aif / AIF_func
    patlak_y_func = resp_func / AIF_func

    patlak_y_func_numeric = lambdify(t, patlak_y_func, modules=["numpy"])
    patlak_t_func_numeric = lambdify(t, patlak_t_func, modules=["numpy"])

    fig_lin, ax_lin = plt.subplots(1, 1, figsize=(5, 5), tight_layout=True)
    ax_lin.plot(
        patlak_t_func_numeric(t_values[1:]), patlak_y_func_numeric(t_values[1:])
    )
    ax_lin.plot(
        patlak_t_func_numeric(frame_times), patlak_y_func_numeric(frame_times), "."
    )
    ax_lin.grid(ls=":")
    ax_lin.set_xlabel("Patlak time")
    ax_lin.set_ylabel("Patlak y")
    ax_lin.set_title("Patlak plot")
else:
    logan_y_func = time_integrated_resp_func / resp_func
    logan_t_func = time_integrated_aif / resp_func

    logan_y_func_numeric = lambdify(t, logan_y_func, modules=["numpy"])
    logan_t_func_numeric = lambdify(t, logan_t_func, modules=["numpy"])

    fig_lin, ax_lin = plt.subplots(1, 1, figsize=(3, 5), tight_layout=True)
    ax_lin.plot(logan_t_func_numeric(t_values[1:]), logan_y_func_numeric(t_values[1:]))
    ax_lin.plot(
        logan_t_func_numeric(frame_times), logan_y_func_numeric(frame_times), "."
    )
    ax_lin.set_aspect(1)
    ax_lin.grid(ls=":")
    ax_lin.set_xlabel("Logan time")
    ax_lin.set_ylabel("Logan y")
    ax_lin.set_title("Logan plot")